In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install cornac

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 19.5 MB/s eta 0:00:00


In [ ]:
!pip install recommenders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.0/356.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.7/750.7 kB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 13.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 75.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 919.8/919.8 kB 48.5 MB/s eta 0:00:00
     ━━━

In [ ]:
import os
import sys
import cornac
import pandas as pd

# from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_random_split
from recommenders.evaluation.python_evaluation import map, ndcg_at_k, precision_at_k, recall_at_k
from recommenders.models.cornac.cornac_utils import predict_ranking
from recommenders.utils.timer import Timer
from recommenders.utils.constants import SEED
from recommenders.utils.notebook_utils import store_metadata

In [ ]:

df = pd.read_csv('/content/drive/MyDrive/data1/df_movie_sen_score.csv')

In [ ]:
df.shape

(3408438, 3)

In [ ]:
data = df[['userID', 'itemID', 'rating']].to_records(index=False)
data_tuples = list(data)

In [ ]:
# from cornac.hyperopt import Discrete, Continuous
# from cornac.hyperopt import GridSearch, RandomSearch

# # Grid Search
# gs_bpr = GridSearch(
#     model=bpr,
#     space=[
#         Discrete(name="k", values=[50,100, 150, 200]),
#         Discrete(name="learning_rate", values=[0.001, 0.05, 0.01, 0.1])
#     ],
#     metric=rec10,
#     eval_method=rs,
# )

In [ ]:
# cornac.Experiment(
#     eval_method=rs,
#     models=[gs_bpr],
#     metrics=[rec10,prc10,ndcg10,mse,rmse,map],
#     user_based=False,
# ).run()

In [ ]:
# print(gs_bpr.best_params)

In [ ]:
from cornac.eval_methods import RatioSplit
from cornac.models import BPR
from cornac.metrics import Precision, Recall, NDCG
from cornac.experiment import Experiment

rs = RatioSplit(data=data_tuples, test_size=0.1, val_size=0.1, rating_threshold=4.0, seed=123)

/usr/local/lib/python3.10/dist-packages/cornac/data/dataset.py:339: UserWarning: 82741 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)
/usr/local/lib/python3.10/dist-packages/cornac/data/dataset.py:339: UserWarning: 1336 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)
/usr/local/lib/python3.10/dist-packages/cornac/data/dataset.py:339: UserWarning: 1331 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


In [ ]:

prc10 = cornac.metrics.Precision(10)
rec10 = cornac.metrics.Recall(10)
ndcg10 = cornac.metrics.NDCG(10)
mse = cornac.metrics.MSE()
rmse = cornac.metrics.RMSE()
map = cornac.metrics.MAP()

metrics=[rec10,prc10,ndcg10,mse,rmse,map]
bpr = BPR(k=50, max_iter=2000, learning_rate=0.1, lambda_reg=0.01, seed=123)

In [ ]:
bpr.fit(rs.train_set)


experiment = Experiment(eval_method=rs, models=[bpr], metrics=metrics)
experiment.run()


VALIDATION:
...
    |    MSE |   RMSE |    MAP | NDCG@10 | Precision@10 | Recall@10 |  Time (s)
--- + ------ + ------ + ------ + ------- + ------------ + --------- + ---------
BPR | 4.4559 | 1.8336 | 0.0541 |  0.0628 |       0.0153 |    0.1048 | 3397.8498

TEST:
...
    |    MSE |   RMSE |    MAP | NDCG@10 | Precision@10 | Recall@10 | Train (s) |  Test (s)
--- + ------ + ------ + ------ + ------- + ------------ + --------- + --------- + ---------
BPR | 4.4733 | 1.8386 | 0.0537 |  0.0621 |       0.0151 |    0.1030 | 3365.7504 | 3514.6071



In [ ]:
# Retrieve user embeddings
user_embeddings = bpr.u_factors

# Retrieve item embeddings
item_embeddings = bpr.i_factors

print("User Embeddings:")
print(user_embeddings)

print("Item Embeddings:")
print(item_embeddings)

User Embeddings:
[[-0.12537403  0.36725235  0.40029302 ... -0.09743519 -0.38285002
  -0.5585562 ]
 [ 0.17209235 -0.09479326  0.21627188 ... -0.02751145 -0.18654521
  -0.5908602 ]
 [ 0.3467806   0.28841043  0.56582075 ... -0.11934241  0.1740919
  -0.21296407]
 ...
 [-0.19275118 -0.14048854  0.65806514 ... -0.17032273  0.15043345
   0.23115791]
 [ 0.09416375  0.16461708  0.40625936 ...  0.3578934  -0.17342016
  -0.34266508]
 [-0.03438398 -0.2000801  -0.12205967 ... -0.12882374 -0.21087036
  -0.08732186]]
Item Embeddings:
[[-0.04189854  0.16868174 -0.4559529  ... -0.24028093 -0.42189562
  -0.3546358 ]
 [-0.02006651 -0.38744166  0.53763247 ... -0.21025904 -0.32787457
  -0.4830283 ]
 [ 0.62199146 -0.00534404  0.8686444  ... -0.02698633 -0.12494618
  -0.49195695]
 ...
 [-0.09244734  0.07424917  0.1382937  ... -0.0099635  -0.13856553
   0.02171327]
 [ 0.07263108  0.11030727  0.13625954 ... -0.08559154 -0.04868526
   0.23858026]
 [ 0.05928699  0.08516433 -0.04998553 ... -0.06313529 -0.07840521

In [ ]:

user_df = pd.DataFrame(user_embeddings)
user_df.index = rs.train_set.user_ids
user_df.columns = [f"factor_{i}" for i in range(user_df.shape[1])]

# Add the user IDs as the first column
user_df.insert(0, "userID", user_df.index)

print("User Embeddings DataFrame:")
print(user_df.head())

User Embeddings DataFrame:
        userID  factor_0  factor_1  factor_2  factor_3  factor_4  factor_5  \
75672    75672 -0.125374  0.367252  0.400293 -0.515595  0.703707 -0.245103   
155986  155986  0.172092 -0.094793  0.216272  0.132569 -0.082528  0.506493   
295918  295918  0.346781  0.288410  0.565821 -0.493932  0.106702 -0.096927   
19232    19232  0.579996 -0.848249  0.182740  0.246303 -0.091540  0.559562   
114581  114581  0.480544 -1.256461  0.113563 -0.444389  0.140337  0.241901   

        factor_6  factor_7  factor_8  ...  factor_40  factor_41  factor_42  \
75672  -0.223416  0.271115  0.567553  ...  -0.310318   0.281762  -0.746162   
155986  0.448314 -0.332091  0.121064  ...   0.555489  -0.399758  -0.231208   
295918 -0.332204  0.098835 -0.045884  ...   0.133377   0.320997  -0.139620   
19232  -0.323912 -0.411598  1.313532  ...   0.524530   0.008207  -0.215204   
114581 -0.368272  0.441459  0.223881  ...   0.408753   0.096963   0.194536   

        factor_43  factor_44  facto

In [ ]:
user_df

,userID,factor_0,factor_1,factor_2,factor_3,factor_4,factor_5,factor_6,factor_7,factor_8,...,factor_40,factor_41,factor_42,factor_43,factor_44,factor_45,factor_46,factor_47,factor_48,factor_49
75672,75672,-0.125374,0.367252,0.400293,-0.515595,0.703707,-0.245103,-0.223416,0.271115,0.567553,...,-0.310318,0.281762,-0.746162,-0.283181,-0.630992,-0.289497,-0.635093,-0.097435,-0.382850,-0.558556
155986,155986,0.172092,-0.094793,0.216272,0.132569,-0.082528,0.506493,0.448314,-0.332091,0.121064,...,0.555489,-0.399758,-0.231208,0.166022,-0.181397,0.128108,-0.183306,-0.027511,-0.186545,-0.590860
295918,295918,0.346781,0.288410,0.565821,-0.493932,0.106702,-0.096927,-0.332204,0.098835,-0.045884,...,0.133377,0.320997,-0.139620,-0.119632,0.181469,0.125166,-0.645389,-0.119342,0.174092,-0.212964
19232,19232,0.579996,-0.848249,0.182740,0.246303,-0.091540,0.559562,-0.323912,-0.411598,1.313532,...,0.524530,0.008207,-0.215204,-0.557532,-0.286729,-0.402904,-0.192539,0.382638,0.570949,-0.474491
114581,114581,0.480544,-1.256461,0.113563,-0.444389,0.140337,0.241901,-0.368272,0.441459,0.223881,...,0.408753,0.096963,0.194536,-0.673262,0.105571,0.534589,-0.322508,-0.100724,-0.050431,0.279671
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32356,32356,-0.233682,0.008842,-0.097980,-0.499756,0.189701,0.220118,-0.123979,0.004634,0.160847,...,0.415191,0.180409,-0.141930,0.082824,-0.042258,0.258978,-0.011097,0.110816,0.498259,-0.149850
292585,292585,-0.040663,0.177471,0.080170,-0.396501,0.286675,0.101812,-0.100003,-0.108072,0.440688,...,0.435437,0.072676,-0.225875,0.094580,-0.042495,0.223933,-0.005932,0.136270,0.501541,0.168523
301802,301802,-0.192751,-0.140489,0.658065,-0.275636,0.203173,0.267184,0.200725,0.015589,-0.416329,...,0.504964,-0.572432,-0.277126,-0.013826,0.526989,-0.063038,0.156379,-0.170323,0.150433,0.231158
237216,237216,0.094164,0.164617,0.406259,-0.174654,0.117646,0.002434,-0.017802,0.119042,0.688852,...,0.197288,-0.163994,0.132501,-0.202057,0.304765,-0.502064,0.212352,0.357893,-0.173420,-0.342665


In [ ]:

item_df = pd.DataFrame(item_embeddings)
item_df.index = rs.train_set.item_ids
item_df.columns = [f"factor_{i}" for i in range(item_df.shape[1])]

# Add the item IDs as the first column
item_df.insert(0, "itemID", item_df.index)

print("Item Embeddings DataFrame:")
print(item_df.head())



Item Embeddings DataFrame:
       itemID  factor_0  factor_1  factor_2  factor_3  factor_4  factor_5  \
46541   46541 -0.041899  0.168682 -0.455953 -0.276489  0.060714  0.476541   
41490   41490 -0.020067 -0.387442  0.537632  0.192597  0.270281  0.424247   
47813   47813  0.621991 -0.005344  0.868644 -0.554553 -0.077077 -0.087551   
14960   14960 -0.238799  0.025475 -0.061057  0.260967 -0.165794  0.078533   
51879   51879  0.151087 -0.586519  0.266499 -0.105788  0.276279 -0.300422   

       factor_6  factor_7  factor_8  ...  factor_40  factor_41  factor_42  \
46541  0.029557 -0.145781 -0.271566  ...   0.184003  -0.274247  -0.003859   
41490  0.151921  0.099387  0.124002  ...   0.433613  -0.047906  -0.265518   
47813 -0.002681  0.242300 -0.155813  ...   0.215384   0.047031   0.267822   
14960 -0.232744 -0.480078  0.436540  ...   0.051726  -0.264938  -0.176372   
51879 -0.014566  0.550291  0.067214  ...   0.303228  -0.222781   0.183265   

       factor_43  factor_44  factor_45  factor_

In [ ]:
user_df

,userID,factor_0,factor_1,factor_2,factor_3,factor_4,factor_5,factor_6,factor_7,factor_8,...,factor_40,factor_41,factor_42,factor_43,factor_44,factor_45,factor_46,factor_47,factor_48,factor_49
75672,75672,-0.125374,0.367252,0.400293,-0.515595,0.703707,-0.245103,-0.223416,0.271115,0.567553,...,-0.310318,0.281762,-0.746162,-0.283181,-0.630992,-0.289497,-0.635093,-0.097435,-0.382850,-0.558556
155986,155986,0.172092,-0.094793,0.216272,0.132569,-0.082528,0.506493,0.448314,-0.332091,0.121064,...,0.555489,-0.399758,-0.231208,0.166022,-0.181397,0.128108,-0.183306,-0.027511,-0.186545,-0.590860
295918,295918,0.346781,0.288410,0.565821,-0.493932,0.106702,-0.096927,-0.332204,0.098835,-0.045884,...,0.133377,0.320997,-0.139620,-0.119632,0.181469,0.125166,-0.645389,-0.119342,0.174092,-0.212964
19232,19232,0.579996,-0.848249,0.182740,0.246303,-0.091540,0.559562,-0.323912,-0.411598,1.313532,...,0.524530,0.008207,-0.215204,-0.557532,-0.286729,-0.402904,-0.192539,0.382638,0.570949,-0.474491
114581,114581,0.480544,-1.256461,0.113563,-0.444389,0.140337,0.241901,-0.368272,0.441459,0.223881,...,0.408753,0.096963,0.194536,-0.673262,0.105571,0.534589,-0.322508,-0.100724,-0.050431,0.279671
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32356,32356,-0.233682,0.008842,-0.097980,-0.499756,0.189701,0.220118,-0.123979,0.004634,0.160847,...,0.415191,0.180409,-0.141930,0.082824,-0.042258,0.258978,-0.011097,0.110816,0.498259,-0.149850
292585,292585,-0.040663,0.177471,0.080170,-0.396501,0.286675,0.101812,-0.100003,-0.108072,0.440688,...,0.435437,0.072676,-0.225875,0.094580,-0.042495,0.223933,-0.005932,0.136270,0.501541,0.168523
301802,301802,-0.192751,-0.140489,0.658065,-0.275636,0.203173,0.267184,0.200725,0.015589,-0.416329,...,0.504964,-0.572432,-0.277126,-0.013826,0.526989,-0.063038,0.156379,-0.170323,0.150433,0.231158
237216,237216,0.094164,0.164617,0.406259,-0.174654,0.117646,0.002434,-0.017802,0.119042,0.688852,...,0.197288,-0.163994,0.132501,-0.202057,0.304765,-0.502064,0.212352,0.357893,-0.173420,-0.342665


In [ ]:
item_df

,itemID,factor_0,factor_1,factor_2,factor_3,factor_4,factor_5,factor_6,factor_7,factor_8,...,factor_40,factor_41,factor_42,factor_43,factor_44,factor_45,factor_46,factor_47,factor_48,factor_49
46541,46541,-0.041899,0.168682,-0.455953,-0.276489,0.060714,0.476541,0.029557,-0.145781,-0.271566,...,0.184003,-0.274247,-0.003859,-0.227370,-0.200421,0.503205,0.100255,-0.240281,-0.421896,-0.354636
41490,41490,-0.020067,-0.387442,0.537632,0.192597,0.270281,0.424247,0.151921,0.099387,0.124002,...,0.433613,-0.047906,-0.265518,0.388657,-0.156926,0.395459,-0.277169,-0.210259,-0.327875,-0.483028
47813,47813,0.621991,-0.005344,0.868644,-0.554553,-0.077077,-0.087551,-0.002681,0.242300,-0.155813,...,0.215384,0.047031,0.267822,-0.459095,-0.214347,-0.282683,-0.712765,-0.026986,-0.124946,-0.491957
14960,14960,-0.238799,0.025475,-0.061057,0.260967,-0.165794,0.078533,-0.232744,-0.480078,0.436540,...,0.051726,-0.264938,-0.176372,-0.064819,-0.121753,0.129832,-0.334616,-0.234501,-0.533550,-0.140289
51879,51879,0.151087,-0.586519,0.266499,-0.105788,0.276279,-0.300422,-0.014566,0.550291,0.067214,...,0.303228,-0.222781,0.183265,-0.690211,0.068105,0.032755,-0.239080,-0.330994,0.038904,0.493336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67609,67609,-0.121473,0.025026,0.139235,0.283376,-0.142907,-0.100976,0.232663,0.121137,0.024522,...,-0.143823,-0.068676,-0.072666,-0.102833,0.088171,-0.156608,0.148466,-0.041113,0.068543,0.312664
65216,65216,-0.080692,0.054294,-0.031908,0.148325,-0.058986,-0.182413,0.176422,0.027527,-0.187522,...,-0.226856,-0.142628,0.019232,-0.196977,0.139308,-0.201609,-0.074308,-0.231342,-0.263349,0.087515
70842,70842,-0.092447,0.074249,0.138294,0.269674,-0.121753,-0.274906,0.178374,-0.016706,0.017102,...,-0.565321,-0.016615,0.035496,0.101633,0.125204,-0.182498,-0.129130,-0.009963,-0.138566,0.021713
70331,70331,0.072631,0.110307,0.136260,0.144572,-0.040157,-0.288172,0.115641,-0.028471,-0.301171,...,-0.369726,-0.263986,-0.108499,0.082562,-0.050308,-0.302964,0.181295,-0.085592,-0.048685,0.238580


In [ ]:

# Export user embeddings
user_embeddings_df = pd.DataFrame(user_df)
user_embeddings_df.to_csv('/content/drive/MyDrive/data2/user_embeddings_movie_sen50.csv', index=False)

# Export item embeddings
item_embeddings_df = pd.DataFrame(item_df)
item_embeddings_df.to_csv('/content/drive/MyDrive/data2/item_embeddings_movie_sen50.csv', index=False)

In [ ]:

import pandas as pd

df_movie = pd.read_csv('/content/drive/MyDrive/data1/df_movie_combined.csv')
df_user_embed = pd.read_csv('/content/drive/MyDrive/data2/user_embeddings_movie_combined.csv')
df_item_embed = pd.read_csv('/content/drive/MyDrive/data2/item_embeddings_movie_combined.csv')

In [ ]:

df_movie = df_movie.dropna()


In [ ]:
df_user_embed.shape

(297462, 101)

In [ ]:
user_movie = df_movie['userID'].unique()
num_users = len(user_movie)
num_users

297492

In [ ]:
df_movie

,userID,itemID,rating
0,16562,11793,5.0
1,16563,11793,5.0
2,12185,11793,5.0
3,16564,11794,5.0
4,16565,11794,5.0
...,...,...,...
3408433,170654,63678,3.5
3408434,43415,63678,4.5
3408435,124234,63678,3.0
3408436,193624,63678,5.0
